# [Project] 텍스트 데이터 기반 문서 분류 프로젝트

## 프로젝트 목표
- 한국어 원문 데이터(법원 판결문)의 요약문을 카테고리('일반행정', '세무', '특허', '형사', '민사', '가사')별로 분류하는 프로젝트 수행

## 데이터 출처
-  https://aihub.or.kr/aidata/8054

## 프로젝트 개요

이번 프로젝트에서는 LSTM 기술을 활용하여 법원 판결문을 분류하는 프로젝트를 수행합니다. 

법원 판결문 데이터를 형태소 분석기를 활용하여 한국어 텍스트를 전처리하는 방법과 이를 학습하여 분류 성능을 확인합니다.

## 0. 필요한 라이브러리 임포트

In [ ]:
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from konlpy.tag import Okt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

## 1. 데이터 전처리

### 데이터 읽기

pandas를 사용하여 `project_data_all3.json` 데이터를 읽고 dataframe 형태로 저장해 봅시다.

In [ ]:
# 엘리스 환경에서 한글 폰트를 사용하기 위한 코드입니다.
font_dirs = ["/usr/share/fonts/truetype/nanum"]
font_files = fm.findSystemFonts(fontpaths=font_dirs)

for font_file in font_files:
    fm.fontManager.addfont(font_file)

plt.rc("font", family="NanumBarunGothic")

# PyTorch에서 사용할 장비 설정 (GPU 사용 가능 시 GPU 사용)  
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  

In [ ]:
# 데이터 파일 경로 정의하기
fp = "./project_data_all3.json"
df = pd.read_json(fp)

### 카테고리 분포 확인

그래프를 활용하여 카테고리 별로 데이터 분포를 확인합니다.

In [ ]:
# 카테고리 분포 확인
plt.figure(figsize=(8, 4))
sns.countplot(data=df, x="category")
plt.xlabel("Category")
plt.ylabel("Number of Documents")
plt.title("Distribution of Document Categories")
plt.xticks(rotation=45)
plt.show()

위 그래프를 확인하면, 형사와 민사가 가장 많고 특허나 가사 요약문은 상대적으로 적다는 것을 확인할 수 있습니다. 

그렇기 때문에 데이터를 학습하기 위하여 테스트셋을 분류할 때 비율을 유지해주는 것이 좋습니다. 

### 카테고리 정수 인코딩

카테고리를 정답 레이블로 활용하기 위하여 숫자데이터로 치환합니다.

In [ ]:
# 원본 카테고리를 별도의 열로 저장
df["original_category"] = df["category"]

# 카테고리를 숫자로 인코딩
label_encoder = LabelEncoder()
df["category"] = label_encoder.fit_transform(df["category"])

### 데이터 전처리

학습에 사용하기 위해 데이터 전처리를 진행합니다.

(약 1분정도 소요됩니다.)

In [ ]:
okt = Okt()


def tokenize(text):
    return okt.nouns(text)


df["abstractive"] = df["abstractive"].apply(lambda x: tokenize(x[0]))

단어 집합 생성 및 단어-인덱스 매핑을 진행합니다.

In [ ]:
word_set = set()
for abstractive in df["abstractive"]:
    word_set.update(abstractive)
word_to_index = {word: index + 1 for index, word in enumerate(word_set)}
word_to_index["<PAD>"] = 0  # 패딩용 토큰

데이터를 텐서로 변환하는 함수를 정의합니다.

In [ ]:
def encode_sentences(sentences, word_to_index, max_len):
    encoded = np.zeros((len(sentences), max_len), dtype=int)
    for i, sentence in enumerate(sentences):
        encoded_sentence = [
            word_to_index[word] for word in sentence if word in word_to_index
        ][:max_len]
        encoded[i, : len(encoded_sentence)] = encoded_sentence
    return torch.tensor(encoded)

데이터를 전처리합니다.

In [ ]:
max_len = 40  # 최대 길이 설정

X = encode_sentences(df["abstractive"].values, word_to_index, max_len)
y = torch.tensor(df["category"].values)

### 데이터 분할 및 데이터 로더 생성

In [ ]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# 데이터 로더 생성
train_dataset = torch.utils.data.TensorDataset(X_train.to(device), y_train.to(device))
test_dataset = torch.utils.data.TensorDataset(X_test.to(device), y_test.to(device))

batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## 2. 모델 정의

### LSTM 모델 정의  

In [ ]:
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, text):
        embedded = self.embedding(text)
        output, (hidden, _) = self.lstm(embedded)
        hidden = hidden.squeeze(0)
        out = self.fc(hidden)
        return out

### 모델 초기화 및 설정

In [ ]:
vocab_size = len(word_to_index)
embedding_dim = 100
hidden_dim = 128
output_dim = len(label_encoder.classes_)

model = LSTMClassifier(vocab_size, embedding_dim, hidden_dim, output_dim).to(device)

## 3. 모델 학습

### 손실 함수 및 최적화 알고리즘 설정  

In [ ]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters())

### 모델 학습 함수 정의

In [ ]:
def train(model, iterator, optimizer, criterion):
    model.train()
    epoch_loss = 0
    for batch in iterator:
        texts, labels = batch
        optimizer.zero_grad()
        predictions = model(texts)
        loss = criterion(predictions, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(iterator)

### 모델 평가 함수 정의

In [ ]:
# 모델 평가 함수
def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for batch in iterator:
            texts, labels = batch
            predictions = model(texts)
            loss = criterion(predictions, labels)
            epoch_loss += loss.item()
    return epoch_loss / len(iterator)

### 모델 학습 

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, optimizer, criterion)
    test_loss = evaluate(model, test_loader, criterion)
    print(
        f"Epoch: {epoch+1:02}, Train Loss: {train_loss:.3f}, Test Loss: {test_loss:.3f}"
    )

## 4. 제출하기

학습한 모델을 바탕으로 모든 요약문에 대한 예측값을 출력하고자 합니다.

### 예측

In [ ]:
# 모델을 평가 모드로 설정
model.eval()

# 전체 데이터셋에 대한 예측값 저장 리스트
all_predictions = []

# DataLoader를 사용하지 않고 전체 데이터에 대해 예측
with torch.no_grad():
    for texts in X:
        texts = texts.unsqueeze(0).to(device)  # 배치 차원 추가
        output = model(texts)
        prediction = output.argmax(dim=1)
        all_predictions.append(prediction.item())

### 다시 문자열로 변환

In [ ]:
# 예측 결과를 df에 'pred' 열로 추가
df["pred"] = all_predictions

# 숫자로된 카테고리를 다시 문자로 변환하기 위한 매핑 딕셔너리 생성
category_map = {index: label for index, label in enumerate(label_encoder.classes_)}

# 숫자로된 예측 결과를 문자열로 변환
df["pred"] = df["pred"].map(
    {index: label for index, label in enumerate(label_encoder.classes_)}
)

# 요약문 별 실제 카테고리와 예측 결과 출력
for i in range(len(df)):
    print("요약문:")
    print(df["abstractive"].iloc[i])
    print("실제 카테고리: %s, " % df["original_category"].iloc[i])
    print("예측 카테고리: %s \n" % df["pred"].iloc[i])

### 결과 출력

In [ ]:
df_submission = df[["id", "pred"]]
df_submission

### 파일로 저장하기

In [ ]:
df_submission.to_csv("submission.csv", index=False)

## TODO. 성능 개선하기

위의 모델을 수정하거나 전처리를 추가하여 성능을 개선해보세요.